In [1]:
pwd

u'/SNS/users/lj7/notebooks/.rk-work/jnrk-sns-analysis-python2-default'

In [154]:
workdir = '/SNS/users/lj7/simulations/10percent scatterer/vanadium'
!mkdir -p "{workdir}"
%cd "{workdir}"

/SNS/users/lj7/simulations/10percent scatterer/vanadium


# Tools

In [3]:
import os, sys, numpy as np, glob
import mcvine
import histogram.hdf as hh, histogram as H

# Simulation app

In [58]:
!mcvine-create-instrument-simulation-application -name=sssd --components=source,save,sample,detector

application 'sssd' created at '/SNS/users/lj7/simulations/10percent scatterer/try1/sssd'


In [60]:
#!cat sssd

In [72]:
!./sssd --source=Source_simple --save=NeutronToStorage --sample=SampleAssemblyFromXml --detector=NeutronToStorage -h -dump-pml

/SNS/software/miniconda2/envs/py2/lib/python2.7/site-packages/mcni/pyre_support/MpiApplication.py:87: UserWarning: no point to run in parallel mode when using just one node
  warnings.warn("no point to run in parallel mode when using just one node")
------------------------------------------------------------
* Instrument simulation application 'sssd'
------------------------------------------------------------
* Sequence of components:
   [source(sources/Source_simple)] --> [save(monitors/NeutronToStorage)] --> [sample(samples/SampleAssemblyFromXml)] --> [detector(monitors/NeutronToStorage)]
------------------------------------------------------------
* Command:
 $ sssd  \
  --dumpconfiguration=<If set, dump configuration to a pml file> \
  --dumpconfiguration-output=<Output file path of dumped configuration file (pml)> \
  --multiple-scattering=<if true, enable multiple scattering> \
  --dump-instrument=<dump-instrument> \
  --buffer_size=<size of neutron buffer. This is for optimizi

In [62]:
# cat sssd.pml

# Sample

In [13]:
cp -a /SNS/users/linjiao/simulations/ARCS/vanadium/April2014/plate/noms/sampleassembly .

In [14]:
ls sampleassembly/

sampleassembly.xml  V-phonons/  V-scatterer.xml  V.xyz


## Estimate 10% scatterer thickness

From NIST site: https://www.ncnr.nist.gov/resources/n-lengths/elements/v.html

In [38]:
barn = 1e-28
sigma = 5.1 *barn

From https://www.webelements.com/vanadium/crystal_structure.html

In [40]:
AA = 1e-10
a = 3.03 * AA

In [41]:
mu = 2*sigma/(a**3)  # 2: bcc
print mu

36.6667389217


exp(-L*mu) = 1-0.1

In [43]:
cm = 0.01
thickness = -np.log(0.9)/mu /cm
print thickness,'cm'

0.287346294643 cm


## More careful estimate

In [89]:
from mcni.utils import conversion as Conv

In [90]:
Ei = 50
vi = Conv.e2v(Ei)
print vi

3092.8377818


In [96]:
sigma_abs = 5.08/vi*2000*barn
mu_abs = 2*sigma_abs/(a**3)
print mu_abs

23.6177584247


(1-exp(-L*(mu_abs+mu)))*(mu/(mu_abs+mu)) = 0.1

In [97]:
thickness = -np.log(1-0.1*(mu_abs+mu)/mu)/(mu_abs+mu)/cm
print thickness

0.297953119357


## Update sample assembly with the estimated thickness

In [98]:
%%file sampleassembly/sampleassembly.xml
<SampleAssembly name="bcc V powder sample assembly">

  <PowderSample name="V" type="sample">
    <Shape>
      <block width="50*mm" height="50*mm" thickness="2.98*mm" />
    </Shape>
    <Phase type="crystal">
      <ChemicalFormula>V</ChemicalFormula>
      <xyzfile>V.xyz</xyzfile>
    </Phase>
  </PowderSample>

  <LocalGeometer registry-coordinate-system="InstrumentScientist">
    <Register name="V" position="(0,0,0)" orientation="(0,0,0)"/>
  </LocalGeometer>

</SampleAssembly>

Overwriting sampleassembly/sampleassembly.xml


## For simplicity, only compute elastic scattering.

In [110]:
%%file sampleassembly/V-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer mcweights="0, 1, 0">

  <KernelContainer average="yes">

<!--    <Phonon_IncoherentElastic_Kernel dw_core='0.00701434948808*angstrom**2'> -->
    <Phonon_IncoherentElastic_Kernel dw_core='0.0*angstrom**2'>
    </Phonon_IncoherentElastic_Kernel>
<!--
    <Phonon_IncoherentInelastic_Kernel>
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS">
      </LinearlyInterpolatedDOS>
    </Phonon_IncoherentInelastic_Kernel>

    <MultiPhonon_Kernel Qmax="15/angstrom">
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS"/>
    </MultiPhonon_Kernel>
-->
  </KernelContainer>
</homogeneous_scatterer>

Overwriting sampleassembly/V-scatterer.xml


# Run Simulation

In [155]:
# !mcvine-component-info --type=Source_simple

In [111]:
rm -rf out/

In [112]:
%%time
!./sssd \
    --source.height=0.05 --source.width=0.05 --source.radius=0 --source.xw=0.05 --source.yh=0.05 \
    --source.E0=50 --source.dE=0.1 --source.dist=1 \
    --save.path=incident-neutrons \
    --sample.xml=sampleassembly/sampleassembly.xml \
    --detector.path=scattered-neutrons \
    --geometer.source="(0,0,-1),(0,0,0)" \
    --geometer.source="(0,0,-0.5),(0,0,0)" \
    --ncount=1e6 \
    --mpirun.nodes=10 \
    > log.sim 2>err.sim

CPU times: user 341 ms, sys: 97.3 ms, total: 439 ms
Wall time: 14.6 s


In [75]:
# cat log.sim

In [113]:
!mcvine-neutron-storage-total-intensity --filename=./out/scattered-neutrons

4792.93186331


In [114]:
!mcvine-neutron-storage-total-intensity --filename=./out/incident-neutrons

62460.9105464


In [103]:
4638.41887227/62460.9393563

0.07426111294629698

**7.4% scatterer? This only considers single-scattering**

# Enable muliple scattering

In [118]:
rm -rf out/

In [119]:
%%time
!./sssd \
    --source.height=0.05 --source.width=0.05 --source.radius=0 --source.xw=0.05 --source.yh=0.05 \
    --source.E0=50 --source.dE=0.1 --source.dist=1 \
    --save.path=incident-neutrons \
    --sample.xml=sampleassembly/sampleassembly.xml \
    --detector.path=scattered-neutrons \
    --geometer.source="(0,0,-1),(0,0,0)" \
    --geometer.source="(0,0,-0.5),(0,0,0)" \
    --ncount=1e6 \
    --multiple-scattering=on \
    --mpirun.nodes=10 \
    > log.sim 2>err.sim

CPU times: user 1.19 s, sys: 329 ms, total: 1.52 s
Wall time: 54.7 s


In [123]:
!mcvine-neutron-storage-total-intensity --filename=./out/scattered-neutrons
!mcvine-neutron-storage-count-neutrons ./out/scattered-neutrons

623408.014261
10881047


In [124]:
623408.014261/10881047

0.057293017322781525

In [125]:
!mcvine-neutron-storage-total-intensity --filename=./out/incident-neutrons
!mcvine-neutron-storage-count-neutrons  ./out/incident-neutrons

62460.9676592
1000000


In [126]:
62460.9676592/1000000

0.062460967659200006

** When multiple scattering is enabled, mcvine always simulate tranmission as weel. So the following percentage is for scattering+transmission**

In [127]:
0.057293017322781525/0.062460967659200006

0.9172611227444332

** That means we have to find out what is the transmission, and then subtract it from the value above **

** Change kernel to simulate transmission only **

In [128]:
%%file sampleassembly/V-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer mcweights="0, 0, 1">

  <KernelContainer average="yes">

<!--    <Phonon_IncoherentElastic_Kernel dw_core='0.00701434948808*angstrom**2'> -->
    <Phonon_IncoherentElastic_Kernel dw_core='0.0*angstrom**2'>
    </Phonon_IncoherentElastic_Kernel>
<!--
    <Phonon_IncoherentInelastic_Kernel>
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS">
      </LinearlyInterpolatedDOS>
    </Phonon_IncoherentInelastic_Kernel>

    <MultiPhonon_Kernel Qmax="15/angstrom">
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS"/>
    </MultiPhonon_Kernel>
-->
  </KernelContainer>
</homogeneous_scatterer>

Overwriting sampleassembly/V-scatterer.xml


** simulate **

In [129]:
rm -rf out/

In [130]:
%%time
!./sssd \
    --source.height=0.05 --source.width=0.05 --source.radius=0 --source.xw=0.05 --source.yh=0.05 \
    --source.E0=50 --source.dE=0.1 --source.dist=1 \
    --save.path=incident-neutrons \
    --sample.xml=sampleassembly/sampleassembly.xml \
    --detector.path=scattered-neutrons \
    --geometer.source="(0,0,-1),(0,0,0)" \
    --geometer.source="(0,0,-0.5),(0,0,0)" \
    --ncount=1e6 \
    --mpirun.nodes=10 \
    > log.sim 2>err.sim

CPU times: user 369 ms, sys: 107 ms, total: 476 ms
Wall time: 18.4 s


In [131]:
!mcvine-neutron-storage-total-intensity --filename=./out/scattered-neutrons
!mcvine-neutron-storage-count-neutrons ./out/scattered-neutrons

51746.1645752
1000000


In [132]:
!mcvine-neutron-storage-total-intensity --filename=./out/incident-neutrons
!mcvine-neutron-storage-count-neutrons  ./out/incident-neutrons

62460.9936602
1000000


** This is percentage of transmission**

In [133]:
51746.1645752/62460.9936602

0.828455673579406

** This is percentage of scattering**

In [134]:
0.9172611227444332-0.828455673579406

0.0888054491650272

**8.88% scatterer**

# Try again

## To make 10% scatter, increase the thickness

In [137]:
2.98*0.1/0.0888

3.3558558558558556

In [138]:
%%file sampleassembly/sampleassembly.xml
<SampleAssembly name="bcc V powder sample assembly">

  <PowderSample name="V" type="sample">
    <Shape>
      <block width="50*mm" height="50*mm" thickness="3.36*mm" />
    </Shape>
    <Phase type="crystal">
      <ChemicalFormula>V</ChemicalFormula>
      <xyzfile>V.xyz</xyzfile>
    </Phase>
  </PowderSample>

  <LocalGeometer registry-coordinate-system="InstrumentScientist">
    <Register name="V" position="(0,0,0)" orientation="(0,0,0)"/>
  </LocalGeometer>

</SampleAssembly>

Overwriting sampleassembly/sampleassembly.xml


## Scattering only - include multiple scattering

In [139]:
%%file sampleassembly/V-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer mcweights="0, 1, 0">

  <KernelContainer average="yes">

<!--    <Phonon_IncoherentElastic_Kernel dw_core='0.00701434948808*angstrom**2'> -->
    <Phonon_IncoherentElastic_Kernel dw_core='0.0*angstrom**2'>
    </Phonon_IncoherentElastic_Kernel>
<!--
    <Phonon_IncoherentInelastic_Kernel>
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS">
      </LinearlyInterpolatedDOS>
    </Phonon_IncoherentInelastic_Kernel>

    <MultiPhonon_Kernel Qmax="15/angstrom">
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS"/>
    </MultiPhonon_Kernel>
-->
  </KernelContainer>
</homogeneous_scatterer>

Overwriting sampleassembly/V-scatterer.xml


## Run sim

In [140]:
rm -rf out/

In [141]:
%%time
!./sssd \
    --source.height=0.05 --source.width=0.05 --source.radius=0 --source.xw=0.05 --source.yh=0.05 \
    --source.E0=50 --source.dE=0.1 --source.dist=1 \
    --save.path=incident-neutrons \
    --sample.xml=sampleassembly/sampleassembly.xml \
    --detector.path=scattered-neutrons \
    --geometer.source="(0,0,-1),(0,0,0)" \
    --geometer.source="(0,0,-0.5),(0,0,0)" \
    --multiple-scattering=on \
    --ncount=1e6 \
    --mpirun.nodes=10 \
    > log.sim 2>err.sim

CPU times: user 1.11 s, sys: 342 ms, total: 1.45 s
Wall time: 54.9 s


In [142]:
!mcvine-neutron-storage-total-intensity --filename=./out/scattered-neutrons
!mcvine-neutron-storage-count-neutrons ./out/scattered-neutrons

617175.763627
10897951


In [143]:
617175.763627/10897951

0.05663227551922376

In [144]:
!mcvine-neutron-storage-total-intensity --filename=./out/incident-neutrons
!mcvine-neutron-storage-count-neutrons  ./out/incident-neutrons

62460.9624862
1000000


In [145]:
62460.9624862/1000000

0.0624609624862

** Percentage of scattering+transmission **

In [146]:
0.05663227551922376/0.0624609624862

0.9066827225362718

## Transmission only

In [147]:
%%file sampleassembly/V-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer mcweights="0, 0, 1">

  <KernelContainer average="yes">

<!--    <Phonon_IncoherentElastic_Kernel dw_core='0.00701434948808*angstrom**2'> -->
    <Phonon_IncoherentElastic_Kernel dw_core='0.0*angstrom**2'>
    </Phonon_IncoherentElastic_Kernel>
<!--
    <Phonon_IncoherentInelastic_Kernel>
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS">
      </LinearlyInterpolatedDOS>
    </Phonon_IncoherentInelastic_Kernel>

    <MultiPhonon_Kernel Qmax="15/angstrom">
      <LinearlyInterpolatedDOS idf-data-path="V-phonons/DOS"/>
    </MultiPhonon_Kernel>
-->
  </KernelContainer>
</homogeneous_scatterer>

Overwriting sampleassembly/V-scatterer.xml


In [149]:
%%time
!rm -rf out
!./sssd \
    --source.height=0.05 --source.width=0.05 --source.radius=0 --source.xw=0.05 --source.yh=0.05 \
    --source.E0=50 --source.dE=0.1 --source.dist=1 \
    --save.path=incident-neutrons \
    --sample.xml=sampleassembly/sampleassembly.xml \
    --detector.path=scattered-neutrons \
    --geometer.source="(0,0,-1),(0,0,0)" \
    --geometer.source="(0,0,-0.5),(0,0,0)" \
    --multiple-scattering=off \
    --ncount=1e6 \
    --mpirun.nodes=10 \
    > log.sim 2>err.sim

CPU times: user 274 ms, sys: 83.9 ms, total: 358 ms
Wall time: 14 s


In [151]:
!mcvine-neutron-storage-total-intensity --filename=./out/scattered-neutrons
!mcvine-neutron-storage-count-neutrons ./out/scattered-neutrons

50519.1627602
1000000


In [150]:
!mcvine-neutron-storage-total-intensity --filename=./out/incident-neutrons
!mcvine-neutron-storage-count-neutrons  ./out/incident-neutrons

62460.9838328
1000000


In [152]:
50519.1627602/62460.9838328

0.8088115117660216

** Percentage of scattering **

In [153]:
0.9066827225362718-0.8088115117660216

0.09787121077025018

**9.8% scatterer!!!**

**Thickness: 3.36*mm**